### Preprocess the raw data from NOAA
This notebook is setup to take in the CSV from NOAA and remove the unneccasary data. This will also seperate out each station for later positioning.

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
#Getting a list of files in raw data folder
filenames = os.listdir('./full_grid')

In [3]:
header_wanted = [
 'HOURLYVISIBILITY',
 'HOURLYDRYBULBTEMPC',
 'HOURLYWETBULBTEMPC',
 'HOURLYDewPointTempC',
 'HOURLYRelativeHumidity',
 'HOURLYWindSpeed',
 'HOURLYWindGustSpeed',
 'HOURLYStationPressure',
 'HOURLYPressureTendency',
 'HOURLYPressureChange',
 'HOURLYSeaLevelPressure',
 'HOURLYPrecip',
 'HOURLYAltimeterSetting']

In [4]:
usecols = ['DATE','STATION'] + header_wanted

In [5]:
#Loading all files into a pandas Dataframe
tqdm.pandas()
df = pd.concat([pd.read_csv('./full_grid/{}'.format(x), usecols=usecols, low_memory=False) for x in tqdm(filenames)])

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [02:07<00:00,  1.55s/it]


In [6]:
#Getting the station names
stations = pd.read_csv("./stations_unique.csv", usecols = ['STATION_ID','LAT_SCALED', 'LON_SCALED'])
wban_list = stations['STATION_ID'].tolist()
len(wban_list)

406

In [7]:
#removes any letters or oddities that appear with numbers for any of the data
def remove_letters(headers,dataframes):
    for i in tqdm(headers):
        dataframes[i].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
        dataframes[i] = dataframes[i].apply(pd.to_numeric)

In [8]:
remove_letters(header_wanted,df)

100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [1:18:33<00:00, 362.54s/it]


In [9]:
by_station_list = []
# Divides dataframe into smaller dataframes of a single WBAN and stores in a list in order of wban_list
for i in tqdm(wban_list):
    by_station_list.append(df.loc[df.STATION == i])
del df

100%|████████████████████████████████████████████████████████████████████████████████| 406/406 [10:29<00:00,  1.55s/it]


In [10]:
by_station_list[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46719 entries, 109127 to 155845
Data columns (total 15 columns):
STATION                   46719 non-null object
DATE                      46719 non-null object
HOURLYVISIBILITY          43438 non-null float64
HOURLYDRYBULBTEMPC        46196 non-null float64
HOURLYWETBULBTEMPC        44790 non-null float64
HOURLYDewPointTempC       46195 non-null float64
HOURLYRelativeHumidity    46195 non-null float64
HOURLYWindSpeed           44852 non-null float64
HOURLYWindGustSpeed       6657 non-null float64
HOURLYStationPressure     44796 non-null float64
HOURLYPressureTendency    0 non-null float64
HOURLYPressureChange      0 non-null float64
HOURLYSeaLevelPressure    0 non-null float64
HOURLYPrecip              2096 non-null float64
HOURLYAltimeterSetting    46197 non-null float64
dtypes: float64(13), object(2)
memory usage: 5.7+ MB


In [11]:
#Sets the date info as the index
for i in tqdm(range(len(by_station_list))):
    by_station_list[i]['STATION_ID'] = by_station_list[i]['STATION']
    by_station_list[i] = by_station_list[i].set_index(pd.DatetimeIndex(by_station_list[i]['DATE']))

100%|████████████████████████████████████████████████████████████████████████████████| 406/406 [00:21<00:00, 18.75it/s]


In [13]:
by_station_list[0]

,STATION,DATE,HOURLYVISIBILITY,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPC,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,STATION_ID
DATE,,,,,,,,,,,,,,,,
2016-08-01 00:15:00,WBAN:00184,2016-08-01 00:15,1000.0,245.0,24.4,245.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184
2016-08-01 00:35:00,WBAN:00184,2016-08-01 00:35,175.0,242.0,24.4,242.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184
2016-08-01 00:55:00,WBAN:00184,2016-08-01 00:55,400.0,242.0,24.4,242.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184
2016-08-01 01:15:00,WBAN:00184,2016-08-01 01:15,100.0,240.0,23.9,240.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184
2016-08-01 01:35:00,WBAN:00184,2016-08-01 01:35,500.0,240.0,23.9,240.0,100.0,0.0,NaN,3000.0,NaN,NaN,NaN,NaN,3005.0,WBAN:00184
2016-08-01 01:55:00,WBAN:00184,2016-08-01 01:55,700.0,241.0,23.9,241.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184
2016-08-01 02:15:00,WBAN:00184,2016-08-01 02:15,700.0,241.0,23.9,241.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184
2016-08-01 02:35:00,WBAN:00184,2016-08-01 02:35,200.0,236.0,23.3,236.0,100.0,0.0,NaN,2998.0,NaN,NaN,NaN,NaN,3003.0,WBAN:00184
2016-08-01 02:55:00,WBAN:00184,2016-08-01 02:55,50.0,235.0,23.3,235.0,100.0,0.0,NaN,2999.0,NaN,NaN,NaN,NaN,3004.0,WBAN:00184


In [14]:
#Some stations take reading multiple times per hour this reduces to one reading per hour by taking the mean of readings
#in an hour
for i in tqdm(range(len(by_station_list))):
    by_station_list[i] = by_station_list[i].resample('60T').mean()

100%|████████████████████████████████████████████████████████████████████████████████| 406/406 [00:06<00:00, 64.90it/s]


In [15]:
#making a list of filenames by removing the colon 
wban_filename = []
for x in wban_list:
    wban_filename.append(x[:4] + x[5:])

In [16]:
wban_filename[0]

'WBAN00184'

In [17]:
wban_list[0]

'WBAN:00184'

In [18]:
#adding back WBAN station since it was absorbed when data was set as the index
for i in tqdm(range(len(by_station_list))):
    by_station_list[i]['STATION'] = wban_list[i] 

100%|███████████████████████████████████████████████████████████████████████████████| 406/406 [00:00<00:00, 619.85it/s]


In [19]:
len(by_station_list[0])

17520

In [34]:
rows = 24 * 365 * 2 
stations_used = []


In [35]:
for i in tqdm(range(len(wban_filename))):
    if(len(by_station_list[i]) >= rows):
        by_station_list[i].to_csv('../Project Final/Transformation/processed_data/{}.csv'.format(wban_filename[i]))
        stations_used.append(stations.loc[stations.STATION_ID == wban_list[i]])

100%|████████████████████████████████████████████████████████████████████████████████| 406/406 [02:04<00:00,  3.27it/s]


In [42]:
stations_used_df = pd.concat(stations_used)

In [44]:
stations_used_df.to_csv('../Project Final/Transformation/stations_mask.csv')

### Current Issues

1) data is not synced across time-zone and all entries start at midnight local time. 

2) some stations have multiple entries per hour and need to be reduced.

Solutions

Remove rows from data based on timezone to sync times
limit only 1 entry per hour for a station